# Code Buddy: Your Technical Code Assistant
Made by: Guru Deep Singh

Code Buddy is an LLM powered assistant allowing you to ask it questions regarding your coding language issues

In [ ]:
#imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [ ]:
# Importing the class   
from CodeBuddy import CodeBuddy

In [ ]:
# Helper functions for streaming
def stream_markdown(generator):
    buffer = ""
    display_handle = display(Markdown(""), display_id=True)

    for part in generator:
        buffer += str(part)
        cleaned = buffer.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned), display_id=display_handle.display_id)

def stream(generator):
    for part in generator:
        print(part, end="", flush=True)

In [ ]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
# Assistant powered by OpenAI API 
myBuddy = CodeBuddy("openai", "gpt-4o-mini")


In [ ]:
output = myBuddy.run(userPrompt = question, stream=False)
display(Markdown(output))

In [ ]:
stream_markdown(myBuddy.run(userPrompt=question, stream=True))

# Ollama based Code Buddy

In [ ]:
# Assistant powered by Ollama 
myBuddyOllama = CodeBuddy("ollama", "llama3.2")

In [ ]:
# Pulling model
!ollama pull llama3.2

In [ ]:
outputOllama = myBuddyOllama.run(userPrompt = question, stream=False)
display(Markdown(outputOllama))

In [ ]:
stream_markdown(myBuddyOllama.run(userPrompt=question, stream=True))